In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
from keras import datasets, layers, models, losses
from random import choice

In [ ]:
image_data = tf.keras.datasets.cifar10
(train_img, train_labels), (test_img, test_labels) = image_data.load_data()

colors=("r", "g", "b")

# this will read the image passed and return a 3-channel histogram of it
#path = r"test_images/lenna.png"
#path = r"test_images/rgb_test.png"
#path = r"test_images/rgb_test2.webp"
#colormap1 = cv2.imread(path) # stores images in BGR format
#colormap1=cv2.cvtColor(colormap1, cv2.COLOR_BGR2RGB)

colormap1 = train_img[100] #choice(train_img)

chans=cv2.split(colormap1)

_, axs = plt.subplots(1, 4, figsize=(12, 12))
plt.imshow(colormap1)
axs = axs.flatten() # works but vscode marks as an error for some reason
i = 0
for img, ax in zip(chans, axs):
    ax.set_title(colors[i])
    ax.imshow(img)
    i += 1
plt.show()

plt.figure()
plt.title("Color histogram")
plt.xlabel("Bins")
plt.ylabel("Number of pixels")
for (chan, c) in zip(chans, colors):
    hist=cv2.calcHist([chan], [0], None, [256], [0,256])
    plt.plot(hist, color=c)
    plt.xlim([0,256])
plt.show()

In [8]:
# this will divide every image in the training set into their respective channels and store them

results = [] # a tuple in the form (label, image, [channels])

print(f"Dividing {len(train_img)} images into channels...")
for i in range(len(train_img)):
    chans=cv2.split(train_img[i])
    results.append((train_labels[i], train_img[i], chans))


Dividing 50000 images into channels...
